# Homework 2
In this assignment, we will be building a Naïve Bayes classifier and a SVM model for the productivity satisfaction of [the given dataset](https://archive.ics.uci.edu/ml/datasets/Productivity+Prediction+of+Garment+Employees), the productivity of garment employees.

## Background
The Garment Industry is one of the key examples of the industrial globalization of this modern era. It is a highly labour-intensive industry with lots of manual processes. Satisfying the huge global demand for garment products is mostly dependent on the production and delivery performance of the employees in the garment manufacturing companies. So, it is highly desirable among the decision makers in the garments industry to track, analyse and predict the productivity performance of the working teams in their factories. 

## Dataset Attribute Information

1. **date**: Date in MM-DD-YYYY
2. **day**: Day of the Week
3. **quarter** : A portion of the month. A month was divided into four quarters
4. **department** : Associated department with the instance
5. **team_no** : Associated team number with the instance
6. **no_of_workers** : Number of workers in each team
7. **no_of_style_change** : Number of changes in the style of a particular product
8. **targeted_productivity** : Targeted productivity set by the Authority for each team for each day.
9. **smv** : Standard Minute Value, it is the allocated time for a task
10. **wip** : Work in progress. Includes the number of unfinished items for products
11. **over_time** : Represents the amount of overtime by each team in minutes
12. **incentive** : Represents the amount of financial incentive (in BDT) that enables or motivates a particular course of action.
13. **idle_time** : The amount of time when the production was interrupted due to several reasons
14. **idle_men** : The number of workers who were idle due to production interruption
15. **actual_productivity** : The actual % of productivity that was delivered by the workers. It ranges from 0-1.

### Libraries that can be used: numpy, scipy, pandas, scikit-learn, cvxpy, imbalanced-learn
Any libraries used in the discussion materials are also allowed.

#### Other Notes

 - Don't worry about not being able to achieve high accuracy, it is neither the goal nor the grading standard of this assignment. <br >
 - If not specified, you are not required to do hyperparameter tuning, but feel free to do so if you'd like.

#### Trouble Shooting
In case you have trouble installing and using imbalanced-learn(imblearn) <br >
Run the below code cell, then go to the selection bar at top: Kernel > Restart. <br >
Then try `import imblearn` to see if things work. 

In [1]:
# Install a pip package in the current Jupyter kernel
from imblearn.over_sampling import SMOTE
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB, GaussianNB
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
import sys
!{sys.executable} -m pip install imbalanced-learn delayed

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.


# Exercises

## Exercise 1 - General Data Preprocessing (20 points)

Our dataset needs cleaning before building any models. Some of the cleaning tasks are common in general, but depends on what kind of models we are building, sometimes we have to do additional processing. These additional tasks will be mentioned in each of the remaining two exercises later.

Note that **we will be using this processed data from exercise 1 in each of the remaining two exercises**.

For convenience, here are the attributes that we would treat them as **categorical attributes**: `day`, `quarter`, `department`, and `team`. 

 - Drop the column `date`.
 - For each of the categorical attributes, **print out** all the unique elements.
 - For each of the categorical attributes, remap the duplicated items, if you find there are typos or spaces among the duplicated items.
     - For example, "a" and "a " should be the same, so we need to update "a " to be "a".
     - Another example, "apple" and "appel" should be the same, so you should update "appel" to be "apple".
     

 - Create another column named `satisfied` that records the productivity performance. The behavior defined as follows. **This is the dependent variable we'd like to classify in this assignment.**
     - Return True or 1 if `actual_productivity` is equal to or greater than `targeted_productivity`. Otherwise, return False or 0, which means the team fails to meet the expected performance.
 - Drop the columns `actual_productivity` and `targeted_productivity`.


 - Find and **print out** which columns/attributes that have empty vaules, e.g., NA, NaN, null, None.
 - Fill the empty values with 0.


In [2]:
# If put the data(.csv) under the same folder, you could use
df = pd.read_csv("./garments_worker_productivity.csv")
df = df.drop("date",axis=1)
cat_var = ["day","quarter","department","team"]
num_var = ["smv","wip","over_time","incentive","idle_time","idle_men","no_of_style_change","no_of_workers"]

for col in cat_var:
    print(col,":",df[col].unique())

df["department"] = df["department"].replace("sweing","sewing")
df["department"] = df["department"].replace("finishing ", "finishing")
#df["department"].unique()
df["satisfied"] = df["actual_productivity"] >= df["targeted_productivity"]
df = df.drop("actual_productivity", axis=1)
df = df.drop("targeted_productivity", axis=1)
df.head()

# print columns with NA/NaN/null/None and replace all w/ 0
df.columns[df.isnull().any()]
df["wip"] = df["wip"].fillna(0)

day : ['Thursday' 'Saturday' 'Sunday' 'Monday' 'Tuesday' 'Wednesday']
quarter : ['Quarter1' 'Quarter2' 'Quarter3' 'Quarter4' 'Quarter5']
department : ['sweing' 'finishing ' 'finishing']
team : [ 8  1 11 12  6  7  2  3  9 10  5  4]


## Exercise 2 - Naïve Bayes Classifier (40 points in total)

### Exercise 2.1 - Additional Data Preprocessing (10 points)

To build a Naïve Bayes Classifier, we need to further encode our categorical variables.

 - For each of the **categorical attribtues**, encode the set of categories to be **0 ~ (n_classes - 1)**.
     - For example, \["paris", "paris", "tokyo", "amsterdam"\] should be encoded as \[1, 1, 2, 0\].
     - Note that the order does not really matter, i.e., \[0, 0, 1, 2\] also works. But you have to start with 0 in your encodings.
     - You can find information about this encoding in the discussion materials.


 - Split the data into training and testing set with the ratio of 80:20, with the **random_state** or the **random_seed** to be **21**.

In [3]:
# Remember to continue the task with your processed data from Exercise 1
label_encoder = LabelEncoder()
cat_df = df[cat_var]
cat_df = cat_df.join(df["satisfied"])
num_df = df[num_var]
num_df = num_df.join(df["satisfied"])
for col in cat_var:
    cat_df[col] = label_encoder.fit_transform(cat_df[col])
    print(col,":",cat_df[col].unique())
    
cat_train,cat_test,num_train,num_test = train_test_split(cat_df,num_df,test_size = 0.2,random_state = 21)

day : [3 1 2 0 4 5]
quarter : [0 1 2 3 4]
department : [1 0]
team : [ 7  0 10 11  5  6  1  2  8  9  4  3]


### Exercise 2.2 - Naïve Bayes Classifier for Categorical Attributes (15 points)

Use the categorical attributes **only**, please build a Categorical Naïve Bayes classifier that predicts the column `satisfied`. <br >
Report the training result and the testing result using `classification_report`.

In [4]:
# Remember to do this task with your processed data from Exercise 2.1
cat_train_y,cat_test_y = cat_train["satisfied"],cat_test["satisfied"]
cat_train_x,cat_test_x = cat_train.drop("satisfied",axis = 1),cat_test.drop("satisfied",axis = 1)
cat_nb = CategoricalNB()
cat_nb.fit(cat_train_x,cat_train_y)
print("Train Score:\n",classification_report(cat_train_y,cat_nb.predict(cat_train_x)))
print("\n\nTest Score:\n",classification_report(cat_test_y,cat_nb.predict(cat_test_x)))

Train Score:
               precision    recall  f1-score   support

       False       0.55      0.20      0.29       254
        True       0.76      0.94      0.84       703

    accuracy                           0.74       957
   macro avg       0.66      0.57      0.57       957
weighted avg       0.71      0.74      0.70       957



Test Score:
               precision    recall  f1-score   support

       False       0.66      0.28      0.39        68
        True       0.77      0.94      0.85       172

    accuracy                           0.75       240
   macro avg       0.71      0.61      0.62       240
weighted avg       0.74      0.75      0.72       240



### Exercise 2.3 - Naïve Bayes Classifier for Numerical Attributes (15 points)

Use the numerical attributes **only**, please build a Gaussian Naïve Bayes classifier that predicts the column `satisfied`. <br >
Report the training result and the testing result using `classification_report`.

Remember to scale your data. The scaling method is up to you.

In [5]:
# Remember to do this task with your processed data from Exercise 2.1

num_train_y,num_test_y = num_train["satisfied"],num_test["satisfied"]
num_train_x,num_test_x = num_train.drop("satisfied",axis = 1),num_test.drop("satisfied", axis = 1)
gss_nb = GaussianNB()
scaler1 = preprocessing.StandardScaler()
scaler1.fit(num_train_x)
gss_nb.fit(scaler1.transform(num_train_x),num_train_y)
print("Train Score:\n",classification_report(num_train_y,gss_nb.predict(num_train_x)))
print("\n\nTest Score:\n",classification_report(num_test_y,gss_nb.predict(num_test_x)))

Train Score:
               precision    recall  f1-score   support

       False       0.00      0.00      0.00       254
        True       0.73      1.00      0.85       703

    accuracy                           0.73       957
   macro avg       0.37      0.50      0.42       957
weighted avg       0.54      0.73      0.62       957



Test Score:
               precision    recall  f1-score   support

       False       0.00      0.00      0.00        68
        True       0.72      1.00      0.83       172

    accuracy                           0.72       240
   macro avg       0.36      0.50      0.42       240
weighted avg       0.51      0.72      0.60       240



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

## Exercies 3 - SVM Classifier (40 points in total)

### Exercise 3.1 - Additional Data Preprocessing (10 points)

To build a SVM Classifier, we need a different encoding for our categorical variables.

 - For each of the **categorical attribtues**, encode them with **one-hot encoding**.
     - You can find information about this encoding in the discussion materials.


 - Split the data into training and testing set with the ratio of 80:20, with the **random_state** or the **random_seed** to be **21**.


In [6]:
# Remember to continue the task with your processed data from Exercise 1

onehot_encoder = OneHotEncoder(sparse = False)
for col in cat_var:
    label_encoded = label_encoder.fit_transform(df[col])
    label_encoded = label_encoded.reshape(len(label_encoded),1)
    c = onehot_encoder.fit_transform(label_encoded)
    df[col] = c.tolist()
    
y = df["satisfied"]
x = df.drop("satisfied",axis = 1).to_numpy()

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.1,random_state = 21)

### Exercise 3.2 - SVM with Different Kernels (20 points)

Using all the attributes we have, please build a SVM that predicts the column `satisfied`. <br >
Specifically, please 
 - Build one SVM with **linear kernel**.
 - Build another SVM but with **rbf kernel**.
 - Report the training results and the testing results of **both models** using `classification report`.

The kernel is the only setting requirement. <br >
Other hyperparameter tuning is not required. But make sure they are the same in these two SVMs if you'd like to tune the model. In other words, the only difference between the two SVMs is the kernel setting.

Remember to scale your data. The scaling method is up to you.

In [10]:
# Remember to do this task with your processed data from Exercise 3.1

lin_svm1 = SVC(kernel = "linear")
rbf_svm1 = SVC(kernel = "rbf")
scaler2 = preprocessing.StandardScaler()
x_train_sc = scaler2.fit_transform(x_train)

lin_svm1.fit(scaler2.transform(x_train_sc),y_train)
display(pd.DataFrame(lin_svm1.getParams().reshape(1,-1),columns = x_train.columns))
print("Train Score:\n",classification_report(y_train,lin_svm1.predict(scaler2.transform(x_train))))
print("\n\nTest Score:\n",classification_report(y_test,lin_svm1.predict(scaler2.transform(x_test))))

rbf_svm1.fit(scaler2.transform(x_train),y_train)
display(pd.DataFrame(rbf_svm1.getParams().reshape(1,-1),columns = x_train.columns))
print("Train Score:\n",classification_report(y_train,rbf_svm1.predict(scaler2.transform(x_train))))
print("\n\nTest Score:\n",classification_report(y_test,rbf_svm1.predict(scaler2.transform(x_test))))

ValueError: setting an array element with a sequence.

### Exercise 3.3 - SVM with Over-sampling (10 points)
 - For the column `satisfied` in our training set, please **print out** the frequency of each class. 
 - Oversample the training data. ( With random_state or random_seed to be 21, if applicable.)
 - For the column `satisfied` in the oversampled data, **print out** the frequency of each class  again.
 - Re-build the 2 SVMs with the same setting you have in Exercise 3.2, but **use oversampled training data** instead.
     - Do not forget to scale the data first. As always, the scaling method is up to you.
 - Report the training result and the testing result with `classification_report`.

You can use ANY methods listed on [here](https://imbalanced-learn.org/stable/references/over_sampling.html#) such as RandomOverSampler or SMOTE. <br > 
You are definitely welcomed to build your own oversampler. <br >

Note that you do not have to over-sample your testing data.

In [ ]:
# Remember to do this task with your processed data from Exercise 3.1

y_train.value_counts()
smote = SMOTE(random_state = 21)
os_x_train,os_y_train = smote.fit_resample(x_train,y_train)
os_y_train.value_counts()

lin_svm2 = SVC(kernel = "linear")
rbf_svm2 = SVC(kernel = "rbf")
scaler3 = MinMaxScaler()
scaler3.fit(os_x_train)
#.fit(x,y)
lin_svm2.fit(scaler.transform(os_x_train),os_y_train)
rbf_svm2.fit(scaler.transform(os_x_train),os_y_train)
#classification_report(y,x)
print("Train Score (Linear):\n",classification_report(os_y_train,lin_svm2.predict(scaler.transform(os_x_train))))
print("\n\nTest Score (Linear):\n",classification_report(y_test,lin_svm2.predict(scaler.transform(x_test))))

print("Train Score (RBF):\n",classification_report(os_y_train,rbf_svm2.predict(scaler.transform(os_x_train))))
print("\n\nTest Score (RBF):\n",classification_report(y_test,rbf_svm2.predict(scaler.transform(x_test))))